## Rating csv


In [1]:
import pandas as pd
import datetime
import numpy as  np
from tqdm import tqdm
np.random.seed(1337)
with open('./kaggle/rating_train.csv', 'r') as f:
    ls = f.readlines()[1:]
u_map = {}

dates = []
foods = []
users = []
    

with tqdm(total=len(ls)) as pbar:
    for l in ls:
        date_str, user, food = l.strip().split(',')
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
        user, food = int(user), int(food)
        if user not in u_map:
            u_map[user] = []
        u_map[user].append( (date, food) )
        
        dates.append(date)
        foods.append(food)
        users.append(user)
        pbar.update(1)
        

        
food_map = {f:i for i, f in enumerate(set(foods))}
user_map = {u:i for i, u in enumerate(set(users))}



100%|██████████| 2681494/2681494 [00:22<00:00, 121423.66it/s]


### Generate training data and label

In [45]:
### ALS embeddingimport numpy as  np
from scipy.sparse import csr_matrix
from constants import MAX_SEQ_LEN

with tqdm(total=len(u_map)) as pbar:
    
#     split 1/10 for future
    X = {}
    X_len = {}
    
    pairs = []
    for user,v in u_map.items():
        history = v
        x = np.zeros([1, MAX_SEQ_LEN, len(food_map)])
        
        ds = np.array([d for d,f in history])
        fs = np.array([f for d,f in history])
        sorted_idx = np.argsort(ds)
        ds = ds[sorted_idx]
        fd = fs[sorted_idx]

        date_idx = 0
        now_date = ds[0]
        for food, date in zip(fs,ds):
            if date != now_date:
                date_idx+=1
                now_date = date
            key = (user, date_idx, food_map[food])
            X[key] = 1
            pair = (user_map[user], food_map[food])
            pairs.append(pair)
        
        X_len[user] = date_idx
        pbar.update(1)
    rows = []
    cols = []
    pairs = set(pairs)
    for pair in pairs:
        u, f = pair
        rows.append(u)
        cols.append(f)
    data = np.ones([len(rows),])
    R = csr_matrix((data, (rows,cols)), shape=(len(user_map),len(food_map)))
print R.shape, len(X), len(X_len)
print 'Done'


100%|██████████| 2608/2608 [00:01<00:00, 1694.78it/s]

(2608, 5532)
Done


In [ ]:
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.approximate_als import (AnnoyAlternatingLeastSquares, FaissAlternatingLeastSquares,
                                      NMSLibAlternatingLeastSquares)
from implicit.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
# initialize a model
MODELS = {"als":  AlternatingLeastSquares,
#           "nmslib_als": NMSLibAlternatingLeastSquares,
#           "annoy_als": AnnoyAlternatingLeastSquares,
#           "faiss_als": FaissAlternatingLeastSquares,
#           "tfidf": TFIDFRecommender,
#           "cosine": CosineRecommender,
          "bpr": BayesianPersonalizedRanking,
#           "bm25": BM25Recommender
         }


def get_model(model_name):
    print("getting model %s" % model_name)
    model_class = MODELS.get(model_name)
    if not model_class:
        raise ValueError("Unknown Model '%s'" % model_name)

    # some default params
    if issubclass(model_class, AlternatingLeastSquares):
        params = {'factors': 128, 'dtype': np.float32, 'iterations':100}
    elif model_name == "bpr":
        params = {'factors': 64, 'dtype': np.float32, 'iterations':100, 'verify_negative_samples':True}
    elif model_name == "bm25":
        params = {'K1': 100, 'B': 0.5}
    elif model_name == 'cosine':
        params = {}
    if model_name == 'faiss_als':
        params['use_gpu'] = False
    return model_class(**params)
# model = implicit.als.AlternatingLeastSquares(factors=256, use_gpu=True, iterations=1000)
rev = {v:k for k,v in user_map.items()}

# for s in MODELS:
for f in [32*2**i for i in range(6)]:
    for it in [10*(i+1) for i in range(10)]:
#     model = get_model(s)
    
        model = MODELS.get('bpr')(use_gpu=True, factors=f, dtype=np.float32, iterations=it, verify_negative_samples=True)
        user_items = R.transpose()
        model.fit(user_items, show_progress=False)
        hits = []
        rec_count = 20
        for idx in range(12,64):
            user = rev[idx]
            recommendations = model.recommend(idx, user_items, N=rec_count)
            last_idx = X_len[user]
            hit = [(user, last_idx, i) in X for i,_ in recommendations]
            hits.append(np.mean(hit))
            # hit = [arr[i]==1 and Y_map[rev[i]][0,i] == 1for i,_ in recommendations]
        print f, it, s, np.mean(hits)

### SVD

In [46]:
import implicit
from scipy.linalg import svd
# 
# a,src_b,c = svd(R.todense()+0.001)
a,src_b,c = svd(R.todense())

In [38]:
import numpy as np
from numpy import linalg as la
a,b,c = la.svd(R.todense())
print a.shape, b.shape,c.shape

(2608, 2608) (2608,) (5532, 5532)


In [44]:
d_s = 2608

b = src_b
print a.shape, b.shape,c.shape
u = csr_matrix((b, (np.arange(b.shape[0]), np.arange(b.shape[0]))), shape=(a.shape[0],c.shape[0])).todense()
p1 = np.matmul(a,u)
p2 = np.matmul(p1,c)
print p2.shape
k = 20
hits = []
rev = {v:k for k,v in user_map.items()}
for u in range(a.shape[0]):
    arr = np.asarray(p2[u,:]).flatten()
    rank_idx = [i for i in reversed(np.argsort(arr)[-k:])]
    hit = [Y_map[rev[u]][0,i] != 0 for i in rank_idx]
    hits.append(np.mean(hit))
print np.mean(hits)
# for u in range()


(2608, 2608) (2608,) (5532, 5532)
(2608, 5532)
0.1839148773006135


In [50]:
d_s = 2608

b = src_b
print a.shape, b.shape,c.shape
u = csr_matrix((b, (np.arange(b.shape[0]), np.arange(b.shape[0]))), shape=(a.shape[0],c.shape[0])).todense()
p1 = np.matmul(a,u)
p2 = np.matmul(p1,c)
print p2.shape
k = 20
rev_food_map = {v:k for k,v in food_map.items()}
rev = {v:k for k,v in user_map.items()}
with open('./SVD_predict.csv', 'w') as f:
    f.write('userid,foodid\n')
    for u in range(a.shape[0]):
        arr = np.asarray(p2[u,:]).flatten()
        rank_idx = [i for i in reversed(np.argsort(arr)[-k:])]
        s = ''
        for food_idx in rank_idx:
            s += ' %d' % rev_food_map[food_idx]
        user = rev[u]
        f.write('%d,%s\n' % (user,s) )
print 'Done'
# for u in range()


(2608, 2608) (2608,) (5532, 5532)
(2608, 5532)
Done


### ALS embedding Training

In [68]:
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.approximate_als import (AnnoyAlternatingLeastSquares, FaissAlternatingLeastSquares,
                                      NMSLibAlternatingLeastSquares)
from implicit.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
# initialize a model
MODELS = {"als":  AlternatingLeastSquares,
#           "nmslib_als": NMSLibAlternatingLeastSquares,
#           "annoy_als": AnnoyAlternatingLeastSquares,
#           "faiss_als": FaissAlternatingLeastSquares,
#           "tfidf": TFIDFRecommender,
#           "cosine": CosineRecommender,
          "bpr": BayesianPersonalizedRanking,
#           "bm25": BM25Recommender
         }


def get_model(model_name):
    print("getting model %s" % model_name)
    model_class = MODELS.get(model_name)
    if not model_class:
        raise ValueError("Unknown Model '%s'" % model_name)

    # some default params
    if issubclass(model_class, AlternatingLeastSquares):
        params = {'factors': 64, 'dtype': np.float32, 'iterations':100}
    elif model_name == "bpr":
        params = {'factors': 512, 'dtype': np.float32, 'iterations':300, 'verify_negative_samples':True}
    elif model_name == "bm25":
        params = {'K1': 100, 'B': 0.5}
    if model_name == 'faiss_als':
        params['use_gpu'] = False
    return model_class(**params)
# model = implicit.als.AlternatingLeastSquares(factors=256, use_gpu=True, iterations=1000)
rev = {v:k for k,v in user_map.items()}

for s in MODELS:
    model = get_model(s)
    user_items = R.transpose()
    model.fit(user_items, show_progress=True)
    hits = []
    kkk=20
    for idx in range(12,64):
        recommendations = model.recommend(idx, user_items, N=kkk)
        hit = [Y_map[rev[idx]][0,i] != 0 for i,_ in recommendations]
        hits.append(np.mean(hit))
        # hit = [arr[i]==1 and Y_map[rev[i]][0,i] == 1for i,_ in recommendations]
    print s, np.mean(hits)

  1%|          | 3/300 [00:00<00:09, 29.76it/s, skipped=22.04%, correct=57.86%]

getting model bpr


100%|██████████| 300/300 [00:08<00:00, 35.40it/s, skipped=22.17%, correct=99.36%]
 14%|█▍        | 14.0/100 [00:00<00:00, 137.86it/s]

bpr 0.0
getting model als


100%|██████████| 100.0/100 [00:00<00:00, 126.71it/s]


als 0.0


In [69]:
model = BayesianPersonalizedRanking(factors=512, use_gpu=True, iterations=100, verify_negative_samples=True)
model.fit(user_items, show_progress=True)
hits = []
kkk=20
for idx in user_map.values():
    recommendations = model.recommend(idx, user_items, N=kkk)
    hit = [Y_map[rev[idx]][0,i] != 0 for i,_ in recommendations]
    hits.append(np.mean(hit))
    # hit = [arr[i]==1 and Y_map[rev[i]][0,i] == 1for i,_ in recommendations]
print s, np.mean(hits)

100%|██████████| 100/100 [00:02<00:00, 34.49it/s, skipped=22.07%, correct=89.77%]


als 0.0


[False, True, True, False, True, True, True, False, False, False, True, False, False, False, False, True, False, True, True, False] 0.45


In [43]:
import implicit

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=256, regularization=0.0001, use_gpu=True, iterations=1000)
# model = implicit.als.AlternatingLeastSquares(factors=16, use_gpu=False)
# train the model on a sparse matrix of item/user/confidence weights
user_items = R.transpose()
model.fit(user_items)
print model.user_factors.shape
print R.shape

idx = 12
kkk=20
print user_items.shape
recommendations = model.recommend(idx, user_items, N=kkk)
print recommendations

# exp = model.explain(idx, user_items, recommendations[0][0], user_weights=None, N=10)
# print exp

arr = np.asarray(R[idx,:].todense()).flatten()
hit = [arr[i]==1 for i,_ in recommendations]
print hit, np.mean(hit)
    


100%|██████████| 1000.0/1000 [00:18<00:00, 53.88it/s]

(2608, 256)
(2608, 5532)
(5532, 2608)
[(168, 1.3599495), (105, 1.3211579), (309, 1.2180662), (530, 1.207345), (86, 1.1754689), (839, 1.1561897), (185, 1.1467923), (272, 1.1443896), (369, 1.141603), (102, 1.1302487), (529, 1.1113687), (213, 1.1074361), (232, 1.1065829), (125, 1.0634801), (42, 1.0571473), (34, 1.0482899), (442, 1.0473948), (166, 1.0442796), (1, 1.0230695), (224, 1.016206)]
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True] 1.0


In [9]:
import implicit
from scipy.linalg import svd

a,b,c = svd(R.todense())


In [15]:
print a.shape, b.shape,c.shape
print np.sort(b)[-1000:]

(2608, 2608) (2608,) (5532, 5532)
[ 10.64648245  10.65367948  10.66624767  10.67021413  10.67334045
  10.68360938  10.68639945  10.70094656  10.70857433  10.7131845
  10.71967545  10.7240006   10.7305354   10.74214919  10.74734611
  10.75218476  10.76177446  10.76757014  10.77296421  10.77622253
  10.78342728  10.79419318  10.79868604  10.80705244  10.81995604
  10.82511747  10.83511752  10.84442115  10.85269664  10.86183542
  10.86247807  10.86922979  10.87368852  10.88398512  10.89248233
  10.89639047  10.90650091  10.91871078  10.92334268  10.93945871
  10.94431328  10.9615918   10.96273193  10.97498591  10.98638158
  10.99363707  10.99600286  11.00331262  11.01178987  11.01769174
  11.0184026   11.0251905   11.03329023  11.04112607  11.04868927
  11.0647313   11.07478709  11.07755246  11.09114625  11.09553927
  11.10346267  11.10827575  11.11706384  11.11965244  11.14115998
  11.14583848  11.15123093  11.16325272  11.16999321  11.1793486
  11.18103032  11.18895846  11.19127406  11.

In [1]:
import torch
a = torch.FloatTensor([0,0,1,1,0,0])
a

In [70]:
# user_items = item_user_data.T.tocsr()
kkk = 20
rev_food_map = {v:k for k,v in food_map.items()}
with open('./ALS_predict.csv', 'w') as f:
    
    f.write('userid,foodid\n')
    with tqdm(total=len(u_map)) as pbar:
        for user, i in user_map.items():
            recommendations = model.recommend(i, user_items, N=kkk)
            s = ''
            for food_idx, value in recommendations:
                s += ' %d' % rev_food_map[food_idx]
            f.write('%d,%s\n' % (user,s) )
            pbar.update(1)
print 'Done'
print len(recommendations)

100%|██████████| 2608/2608 [00:09<00:00, 282.22it/s]

Done
20


In [49]:

print len(recommendations)
idx = 12
print user_items.shape
recommendations = model.recommend(idx, user_items, N=kkk)
print recommendations

exp = model.explain(idx, user_items, recommendations[0][0], user_weights=None, N=10)
# print exp

arr = np.asarray(R[idx,:].todense()).flatten()
hit = [arr[i]==1 for i,_ in recommendations]
print hit, np.mean(hit)
    
# print [(i,arr[i]) for i in reversed(np.argsort(arr)[-kkk:])]

# # find related items
# a = model.user_factors
# b = model.item_factors
# c = np.matmul(a,b.transpose())
# arr = np.asarray(c[idx,:]).flatten()
# print [(i,arr[i]) for i in reversed(np.argsort(arr)[-kkk:])]

20
(5532, 2608)
[(39, 1.0589447), (139, 1.0535698), (18, 1.0461819), (34, 1.0157015), (107, 0.9683679), (168, 0.96572435), (41, 0.9407524), (125, 0.9335048), (839, 0.92211986), (224, 0.9000232), (25, 0.89088106), (88, 0.8828891), (117, 0.83348477), (80, 0.82915735), (763, 0.8249409), (81, 0.8217071), (102, 0.8114673), (105, 0.7993405), (42, 0.7891038), (69, 0.7713862)]
[True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True] 0.95


### ALS testing

In [18]:

print model.user_factors.shape
# print np.matmul(model.user_factors, model.item_factors.transpose()).shape
print R.shape
# for user, i in user_map.items():
    


(5532, 256)
(2608, 5532)


#### prepare sample generator


In [5]:
from constants import MAX_SEQ_LEN

val_num = len(pseudo_history_map) // 10
idx = np.random.permutation(len(pseudo_history_map))
train_idx, val_idx = idx[val_num:], idx[:val_num]
train_u_map = {k:pseudo_history_map[k] for k in pseudo_history_map.keys()[val_num:]}
val_u_map = {k:pseudo_history_map[k] for k in pseudo_history_map.keys()[:val_num]}
def batch_boostrap_generator(batch_size, u_map, food_map, Y_map, max_history_len, flip):
    G = boostrap_generator(u_map, food_map, Y_map, max_history_len, flip)
    while True:
        X = []
        pos_Y = []
        neg_Y = []
        for i in range(batch_size):
            x, pos_y, neg_y = next(G)
            x = np.pad(x, ((0,max_history_len-x.shape[0]),(0,0)), 'constant', constant_values=0)
            X.append(np.expand_dims(x, axis=0))
            pos_Y.append(np.expand_dims(pos_y, axis=0))
            neg_Y.append(np.expand_dims(neg_y, axis=0))
        yield np.vstack(X), np.vstack(pos_Y), np.vstack(neg_Y) 
def boostrap_generator(u_map, food_map, Y_map, max_history_len, flip):
    while True:
        keys = u_map.keys()
        for user_idx in np.random.permutation(len(u_map)):
            user = keys[user_idx]
            X = np.zeros([max_history_len, len(food_map)])
            Y = Y_map[user].flatten()
            history = u_map[user]
            ds = np.array([d for d,f in history])
            fs = np.array([f for d,f in history])
            sorted_idx = np.flip(np.argsort(ds), axis=-1) if flip else np.argsort(ds)
            ds = ds[sorted_idx]
            fd = fs[sorted_idx]
            
            date_idx = 0
            now_date = ds[0]
            for food, date in zip(fs,ds):
                if date != now_date:
                    date_idx+=1
                    now_date = date
                X[date_idx, food_map[food]] = 1
            
    #         positive sample
            idx = np.random.permutation(len(Y))
            pos_i = neg_i = -1
            for i in idx:
                if Y[i] == 1 and pos_i == -1:
                    pos_i = i
                if Y[i] == 0 and neg_i == -1:
                    neg_i = i
                if pos_i != -1 and neg_i != -1:
                    break
            yield X, np.array([pos_i]), np.array([neg_i])
            
    

G = batch_boostrap_generator(32, train_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN, flip=True)
val_G = batch_boostrap_generator(32//2, val_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN, flip=True)

x, pos_y, neg_y = next(G)
print x.shape, pos_y.shape, neg_y.shape
x, pos_y, neg_y = next(val_G)
print x.shape, pos_y.shape, neg_y.shape

G2 = boostrap_generator(train_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN, flip=True)
x, pos_y, neg_y = next(G2)
print x.shape, pos_y.shape, neg_y.shape


(32, 165, 5532) (32, 1) (32, 1)
(16, 165, 5532) (16, 1) (16, 1)
(165, 5532) (1,) (1,)


In [6]:
a = np.array([3,2,1,4])
b = np.flip(a, axis=-1)
b

array([4, 1, 2, 3])

In [ ]:
from collections import deque
from tqdm import tqdm as tqdm

import time
def dump_log(model, n_iter, loss, acc, val_loss, val_acc, log_file_stream, tmp_model_path):
    log_text = '%.7d<split>%.5f<split>%.5f<split>%.5f<split>%.5f\n' % (n_iter, loss, acc, val_loss, val_acc)
    log_file_stream.write(log_text)
    if n_iter % 10 == 0 :
        log_file_stream.flush()
        torch.save(model, tmp_model_path)
        
acc_q = deque(maxlen=1000)
loss_q = deque(maxlen=1000)
val_acc_q = deque(maxlen=1000)
val_loss_q = deque(maxlen=1000)
t = time.time()
best_acc  = 0
best_loss = float('inf')

epochs = 100
batch_size = 64
G = batch_boostrap_generator(batch_size, train_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN, flip=False)
val_G = batch_boostrap_generator(batch_size, val_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN, flip=False)

print 'start training.'
with open('log.txt', 'a') as f:
    with open('best.txt', 'w') as best_log:
        for e in range(epochs):
            iters = len(food_map) * len(user_map)
            with tqdm(total=iters) as pbar:
                for it in range(iters):
                    optimizer.zero_grad()
                    model.train()
                    x, pos_y, neg_y = next(G)
    #                 print 'x1', np.sum(x[0,:])
                    x = torch.FloatTensor(x).cuda()
                    pos_y,neg_y = torch.LongTensor(pos_y), torch.LongTensor(neg_y)

    #                 positive
                    output = model(pos_y.cuda(), x)
                    a = output[0,0].item()
                    pred = output > 0.5
                    label = torch.ones_like(output).cuda()
                    pos_loss = criterion(output, label)

                    acc = torch.sum(pred == label.type(torch.uint8)).item() / float(output.shape[0])
                    acc_q.append(acc)
    #                 pos_loss.backward()

    #                 negative
                    output = model(neg_y.cuda(), x)
                    b = output[0,0].item()
                    pred = output > 0.5
                    label = torch.zeros_like(output).cuda()
                    neg_loss = criterion(output, label )
                    acc = torch.sum(pred == label.type(torch.uint8)).item() / float(output.shape[0])
                    acc_q.append(acc)
    #                 neg_loss.backward()

                    loss = pos_loss + neg_loss
                    loss.backward()
                    optimizer.step()
                    with torch.no_grad():
                        model.eval()
    #                 positive
                        x, pos_y, neg_y = next(val_G)
    #                     print 'x2', np.sum(x[0,:])
                        x = torch.FloatTensor(x).cuda()
                        pos_y,neg_y = torch.LongTensor(pos_y), torch.LongTensor(neg_y)
    #                       
                        output = model(pos_y.cuda(), x)
    #                     o1 = model.transformer.decoder.mid
    #                     o1 = model.mid

                        c = output[0,0].item()
                        pred = output > 0.5
                        label = torch.ones_like(output).cuda()

                        pos_loss = criterion(output, label )

                        val_acc = torch.sum(pred == label.type(torch.uint8)).item() / float(output.shape[0])
                        val_acc_q.append(val_acc)
    #                 negative
    #                     print pos_y, neg_y
    #                     print output

                        output = model(neg_y.cuda(), x)
    #                     o2 = model.mid
    #                     print o2.shape
    #                     print neg_y.cuda().shape
    #                     print neg_y.cuda()[0,0]
    #                     print (o1 == o2).all()
                        d = output[0,0].item()
    #                     print c == d
    #                     wf
    #                     print pos_y == neg_y
    #                     print output


                        pred = output > 0.5
                        label = torch.zeros_like(output).cuda()

                        neg_loss = criterion(output, label )
                        val_acc = torch.sum(pred == label.type(torch.uint8)).item() / float(output.shape[0])
                        val_acc_q.append(val_acc)

                        val_loss = pos_loss + neg_loss
                    acc = np.mean(acc_q)
                    val_acc = np.mean(val_acc_q)

    #                 dump_log(model, it+1, loss, acc, val_loss, val_acc, f, './tmp.pt')

    #                 pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, %.3f, %.3f, %.3f' % (acc, val_acc,a,b,c), refresh=False)
                    pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, %.3f, %.3f, %.5f, %.5f' % (acc, val_acc,a,b,c,d), refresh=False)
                    pbar.update(batch_size)
                    dump_log(model, (it+1)*batch_size, loss, acc, val_loss, val_acc, f, './tmp.pt')
                    if val_acc > best_acc and it > 100:
                        torch.save(model, './best.pt')
                        best_acc = val_acc
                        best_log.write('%d\t%.5f\n' % ((it+1)*batch_size, best_acc))

# Train model
print("Optimization Finished!")
# print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

  0%|          | 0/14427456 [00:00<?, ?it/s]

start training.


  0%|          | 320/14427456 [00:07<88:27:13, 45.31it/s, acc : 0.528, val_acc : 0.525, 0.578, 0.529, 0.42413, 0.50843] /home/k123/env/python2.7.15/lib/python2.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/k123/env/python2.7.15/lib/python2.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Food_embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
  3%|▎         | 372992/14427456 [1:44:16<60:10:55, 64.87it/s, acc : 0.739, val_acc : 0.750, 0.945, 0.603, 0.74552, 0.55210]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
N